In [107]:
import gns3fy
from gns3fy import Gns3Connector
from gns3fy import Node, Link
from threading import Thread
import multiprocessing

In [111]:
server = Gns3Connector('http://localhost:3080')
lab = gns3fy.Project(name='test2', connector=server)
lab.get()

In [3]:
pid = lab.project_id

In [ ]:
lab.status

In [ ]:
lab.links_summary()

In [ ]:
s2 = Node(project_id=pid, name='s2', connector=server, template='Ethernet switch')
s2.create()

In [ ]:
host1 = Node(project_id=pid, name='host1', connector=server, template='VPCS')
host2 = Node(project_id=pid, name='host2', connector=server, template='VPCS')

In [ ]:
# host1.create()
# host2.create()

host1.get()
host2.get()

In [ ]:
host1.ports

In [ ]:
# nodes = [
#     dict(node_id=alpine1.node_id, adapter_number=0, port_number=1),
#     dict(node_id=host1.node_id, adapter_number=0, port_number=0)
# ]

# h1_ap1_link = Link(project_id=pid, nodes=nodes, connector=server)
# h1_ap1_link.create()

In [ ]:
# nodes = [
#     dict(node_id=alpine1.node_id, adapter_number=0, port_number=2),
#     dict(node_id=host2.node_id, adapter_number=0, port_number=0)
# ]

# h2_ap1_link = Link(project_id=pid, nodes=nodes, connector=server)
# h2_ap1_link.create()

In [ ]:
import telnetlib
import time

In [ ]:
h1_console = telnetlib.Telnet('127.0.0.1', 5002, timeout=30)
h1_console.write(b'show ip\n')
data = h1_console.read_until(b'EOF\n', timeout=0.1)
f = open('ips', 'w')
f.write(data.decode('utf-8'))
f.close()
h1_console.close()

In [ ]:
import re
pattern = r'(\d+.\d+.\d+.\d+/\d+)'
match = re.search(pattern, data.decode("utf-8"))
if match:
        print(match.group(1))

In [ ]:
import re
def extract_ip(data):        
    pattern = r'(\d+.\d+.\d+.\d+/\d+)'
    match = re.search(pattern, data.decode("utf-8"))
    if match:
        return match.group(1)
    raise KeyError()

In [ ]:
def get_ip(node) -> None:
    console = telnetlib.Telnet('127.0.0.1', node.console, timeout=5)
    console.write(b'show ip\n')
    data = console.read_until(b'EOF\n', timeout=0.1)
    try:
        ip = extract_ip(data)
    except KeyError:
        print(f'ip not found for node {node.name} with port: {node.console}')

    f = open('ips', 'a')
    f.write(f'{node.name}, {node.console}, {ip}\n')
    f.close()
    console.close()

def set_ip(node, ip_addr, gateway_addr) -> None:
    console = telnetlib.Telnet('127.0.0.1', node.console, timeout=5)
    cmd = f'ip {ip_addr} {gateway_addr}\n'
    console.write(cmd.encode('utf-8'))
    data = console.read_until(b'EOF\n', timeout=0.1)
    console.close()



In [ ]:
pc4 = Node(project_id=pid, name='PC4', connector=server, template='VPCS')
pc4.get()

In [ ]:
set_ip(pc4, '192.168.1.4', '255.0.0.0')

In [ ]:
get_ip(pc4)

In [ ]:
nodes = lab.nodes
for node in nodes:
    if node.node_type == 'vpcs':
        get_ip(node)

##### aight now to build the tree topology : )
##### soooo how would it look like sef

In [ ]:
pc9 = Node(project_id=pid, name='pc9', connector=server, template='VPCS').create()


In [ ]:
lab.get()
lab.nodes

In [ ]:
#create vpcs(es)
for i in range(6):
    Node(project_id=pid, name=f'pc{i}', connector=server, template='VPCS').create()

#create switches
for i in range(2):
    Node(project_id=pid, name=f's{i}', connector=server, template='Ethernet switch').create()



In [ ]:
lab.get()
lab.nodes

In [ ]:
3//3

In [ ]:
s = lab.nodes[6:]
n = lab.nodes[:6]





In [ ]:
for i in range(3):
    Node(project_id=pid, name=f's{i}', connector=server, template='Ethernet switch').create()


In [ ]:
s

In [ ]:
for i in range(len(n)):
    print(i%len(s) + 1)

In [ ]:
for i in range(len(n)):
    port = (i%3) + 1 #3 here shows the number of hosts that would be connected to a switch.
    nodes = [
             dict(node_id=s[i//3].node_id, adapter_number=0, port_number=port),
             dict(node_id=n[i].node_id, adapter_number=0, port_number=0)
            ]
    
    Link(project_id=pid, nodes=nodes, connector=server).create()


In [ ]:
for i in range(len(s)-1):
    port = (i % 2) + 4
    nodes = [
        dict(node_id=s[i].node_id, adapter_number=0, port_number=port),
        dict(node_id=s[i+1].node_id, adapter_number=0, port_number=port),        
    ]
    Link(project_id=pid, nodes=nodes, connector=server).create()


In [9]:
pc1 = lab.nodes[0]
pc1

Node(name='pc1', project_id='e2c54b87-d641-4b2e-aa34-d6173787d1cd', node_id='4132a261-5630-47ad-8771-a3b259f38d53', compute_id='local', node_type='vpcs', node_directory='/home/g1f7/GNS3/projects/e2c54b87-d641-4b2e-aa34-d6173787d1cd/project-files/vpcs/4132a261-5630-47ad-8771-a3b259f38d53', status='started', ports=[{'adapter_number': 0, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'Ethernet0', 'port_number': 0, 'short_name': 'e0'}], port_name_format='Ethernet{0}', port_segment_size=0, first_port_name=None, locked=False, label={'rotation': 0, 'style': None, 'text': 'pc1', 'x': None, 'y': -40}, console=5000, console_host='0.0.0.0', console_type='telnet', console_auto_start=True, command_line='/usr/bin/vpcs -p 5001 -m 0 -i 1 -F -R -s 20000 -c 20001 -t 127.0.0.1 startup.vpc', custom_adapters=[], height=59, width=65, symbol=':/symbols/vpcs_guest.svg', x=0, y=0, z=1, template_id='19021f99-e36f-394d-b4a1-8aaa902ab9cc', properties={}, template=None)

In [13]:
pc1.console_auto_start

True

In [109]:
import re
import telnetlib
import time

def extract_ip(data):        
    pattern = r'(\d+.\d+.\d+.\d+/\d+)'
    match = re.search(pattern, data.decode("utf-8"))
    if not match:
        raise KeyError()
    else:
        return match.group(1)
    

def get_ip(node) -> None:
    console = telnetlib.Telnet('127.0.0.1', node.console, timeout=5)
    cmd = 'show ip\n'
    console.write(cmd.encode('utf-8'))
    data = console.read_until(b'EOF\n', timeout=0.1)
    try:
        ip = extract_ip(data)
    except KeyError:
        print(f'ip not found for node {node.name} with port: {node.console}')
    else:
        print(f'{node.name}, {node.console}, {ip}\n')
    console.close()

def set_ip(node, ip_addr, gateway_addr) -> None:
    console = telnetlib.Telnet('127.0.0.1', node.console, timeout=5)
    cmd = f'ip {ip_addr} {gateway_addr}\n'
    console.write(cmd.encode('utf-8'))
    time.sleep(3)
    console.close()
    


In [112]:
vpcs = lab.nodes[:9]
vpcs

[Node(name='pc1', project_id='9612d158-02dd-4a6a-9acb-652c3a893fa4', node_id='e51edaca-972f-46b5-ab30-4ccf32426e72', compute_id='local', node_type='vpcs', node_directory='/home/g1f7/GNS3/projects/9612d158-02dd-4a6a-9acb-652c3a893fa4/project-files/vpcs/e51edaca-972f-46b5-ab30-4ccf32426e72', status='started', ports=[{'adapter_number': 0, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'Ethernet0', 'port_number': 0, 'short_name': 'e0'}], port_name_format='Ethernet{0}', port_segment_size=0, first_port_name=None, locked=False, label={'rotation': 0, 'style': None, 'text': 'pc1', 'x': None, 'y': -40}, console=5000, console_host='0.0.0.0', console_type='telnet', console_auto_start=True, command_line='/usr/bin/vpcs -p 5001 -m 0 -i 1 -F -R -s 20000 -c 20001 -t 127.0.0.1 startup.vpc', custom_adapters=[], height=59, width=65, symbol=':/symbols/vpcs_guest.svg', x=0, y=0, z=1, template_id='19021f99-e36f-394d-b4a1-8aaa902ab9cc', properties={}, template=None),
 Node(nam

In [113]:
get_ip(vpcs[6])

pc7, 5012, 0.0.0.0/0



In [91]:

addr = f'192.168.1.{vpcs[6].name[-1]}'
gtway = '255.0.0.0'
set_ip(node=vpcs[6], ip_addr=addr, gateway_addr=gtway)

In [114]:
processes = []

#assign ip addresses to vpcs
for pc in vpcs:
    addr = f'192.168.1.{pc.name[-1]}'
    gtway = '255.0.0.0'
    process = multiprocessing.Process(target=set_ip, args=[pc, addr, gtway])
    processes.append(process)
    process.start()

for process in processes:
    process.join()
    
    

In [115]:
for pc in vpcs:
    get_ip(pc)

pc1, 5000, 192.168.1.1/8

pc2, 5002, 192.168.1.2/8

pc3, 5004, 192.168.1.3/8

pc4, 5006, 192.168.1.4/8

pc5, 5008, 192.168.1.5/8

pc6, 5010, 192.168.1.6/8

pc7, 5012, 192.168.1.7/8

pc8, 5014, 192.168.1.8/8

pc9, 5016, 192.168.1.9/8



In [100]:
def test(a):
    time.sleep(2)
    print(a)
ii = [1, 2, 3, 4, 5]

processes = []

for i in ii:
    process = multiprocessing.Process(target=test, args=[i])
    processes.append(process)
    process.start()

print('5 processes started')

# for process in processes:
#     process.join()

print('5 processes finished')

5 processes started
5 processes finished


1
2
3
4
5
